<a href="https://colab.research.google.com/github/charann90/Scraping-Doctor-Information-Empowering-Healthcare-Analysis/blob/main/doctors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [3]:
import csv

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
df=pd.read_csv('/content/pedia1.csv')
len(df)

FileNotFoundError: ignored

In [46]:
df.drop(['Name', 'address', 'Phone_number'], axis=1, inplace=True)

In [47]:
df

,links
2,https://www.yellowpages.com/search?search_term...
3,https://www.yellowpages.com/search?search_term...
4,https://www.yellowpages.com/search?search_term...
5,https://www.yellowpages.com/search?search_term...
6,https://www.yellowpages.com/search?search_term...
...,...
865,https://www.yellowpages.com/search?search_term...
866,https://www.yellowpages.com/search?search_term...
867,https://www.yellowpages.com/search?search_term...
868,https://www.yellowpages.com/search?search_term...


In [27]:
links = df['links']  # Example: Process the first 20 links
batch_size = 5  # Number of links to process in each batch
num_processes = 10  # Number of processes to use

names = []
addresses = []
phone_numbers = []
personal_links = []
categories = []
years = []

# Define a function to process a batch of links
def process_batch(batch_links):
    batch_names = []
    batch_addresses = []
    batch_phone_numbers = []
    batch_personal_links = []
    batch_categories = []
    batch_years = []

    for link in batch_links:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract the doctor's name
        name_element = soup.find("a", class_="business-name")
        name = name_element.text.split(', ')[0] if name_element else ""

        # Extract the address
        address_element = soup.find("div", class_="adr")
        address = address_element.get_text(strip=True) if address_element else ""

        # Extract the phone number
        phone_number_element = soup.find("div", class_="phones phone primary")
        phone_number = phone_number_element.text.strip() if phone_number_element else ""

        # Extract the personal page link
        try:
            personal_page_element = soup.find("div", class_="links")
            page = personal_page_element.find("a")["href"] if personal_page_element else ""
        except (AttributeError, TypeError):
            page = ""

        # Extract the categories
        cat_ele = soup.find("div", class_="categories")
        categories = [category.get_text(strip=True) for category in cat_ele.find_all("a")] if cat_ele else []

        # Extract the years of experience
        year_ele = soup.find("div", class_="number")
        year = year_ele.text if year_ele else ""

        batch_names.append(name)
        batch_addresses.append(address)
        batch_phone_numbers.append(phone_number)
        batch_personal_links.append(page)
        batch_categories.append(categories)
        batch_years.append(int(year) if year else 0)

    return (
        batch_names, batch_addresses, batch_phone_numbers,
        batch_personal_links, batch_categories, batch_years
    )

# Split the links into batches
batches = [links[i:i+batch_size] for i in range(0, len(links), batch_size)]

# Create a ProcessPoolExecutor
with concurrent.futures.ProcessPoolExecutor(max_workers=num_processes) as executor:
    # Submit tasks for each batch of links
    results = [executor.submit(process_batch, batch) for batch in batches]

    # Process the results as they complete
    for future in concurrent.futures.as_completed(results):
        # Retrieve the results of a completed task
        (
            batch_names, batch_addresses, batch_phone_numbers,
            batch_personal_links, batch_categories, batch_years
        ) = future.result()

        # Append the batch results to the respective lists
        names.extend(batch_names)
        addresses.extend(batch_addresses)
        phone_numbers.extend(batch_phone_numbers)
        personal_links.extend(batch_personal_links)
        categories.extend(batch_categories)
        years.extend(batch_years)

print("Names:", names)
print("Addresses:", addresses)
print("Phone Numbers:", phone_numbers)
print("Personal Links:", personal_links)
print("Categories:", categories)
print("Years:", years)


Names: ['', 'Advanced Urology', 'Dr. Yana Tavyev', 'Dr. Yana Tavyev', 'Dr. Philip H. Seifer', 'Benjamin Tiet', 'Benjamin Tiet', 'Dr. Yana Tavyev', 'Kaukauna Clinic', 'Benjamin Tiet', 'Advanced Urology', 'Dr. Lawrence Ian Kagan', 'Eisner Pediatric & Family Medical Center', 'California Kids Pediatrics', 'ABC Pediatrics Inc', 'California Kids Pediatrics', 'Benjamin Tiet', 'Benjamin Tiet', 'Benjamin Tiet', 'Benjamin Tiet', 'Benjamin Tiet', 'Eisner Pediatric & Family Medical Center', 'Dr. Manpreet Virk', 'Eisner Pediatric & Family Medical Center', 'Christine Kao', 'Comprehensive Urology Medical Group', 'Kaukauna Clinic', 'Danelle M. Fisher', 'Danelle M. Fisher', 'Danelle M. Fisher', 'Benjamin Tiet', 'Eisner Pediatric & Family Medical Center', 'Advanced Urology', 'Charlotte Sarshad', 'Benjamin Tiet', 'Christine Kao', 'Advanced Urology', 'Charlotte Sarshad', 'Nicole K. Militello', 'Dr. Yovana Graciela Bruno', 'California Kids Pediatrics', 'Advanced Urology', 'Advanced Urology', 'Advanced Urol

In [41]:
df['links']=df['links'].iloc[1:]

len(df['links'])

870

In [49]:
df['Name']=names
df['Address']=addresses
df['Phone_number']= phone_numbers
df['Categories']= categories
df['Personal_website']=personal_links
df['Years_in_practice']= years

In [50]:
df

,Name,links,address,Phone_number,Address,Categories,Personal_website,Years_in_practice
0,,https://www.yellowpages.com/search?search_term...,"2990 S Sepulveda Blvd Ste 206Los Angeles, CA 9...",,,[],,0
1,Advanced Urology,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(424) 732-1771,"8540 S Sepulveda BlvdLos Angeles, CA 90045","[Physicians & Surgeons, Pediatrics, Medical Ce...",http://www.advancedurology.net,26
2,Dr. Yana Tavyev,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(310) 967-1884,"8700 Beverly BlvdWest Hollywood, CA 90048","[Physicians & Surgeons, Pediatrics, Physicians...",https://www.cedars-sinai.edu,22
3,Dr. Yana Tavyev,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(310) 967-1884,"8700 Beverly BlvdWest Hollywood, CA 90048","[Physicians & Surgeons, Pediatrics, Physicians...",https://www.cedars-sinai.edu,15
4,Dr. Philip H. Seifer,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(323) 857-2000,"6041 Cadillac Ave Ste 451Los Angeles, CA 90034","[Physicians & Surgeons, Pediatrics, Physicians...",https://healthy.kaiserpermanente.org/health/po...,30
...,...,...,...,...,...,...,...,...
865,Advanced Urology,https://www.yellowpages.com/search?search_term...,"1403 Lomita BlvdHarbor City, CA 90710",(424) 732-1771,"8540 S Sepulveda BlvdLos Angeles, CA 90045","[Physicians & Surgeons, Pediatrics, Medical Ce...",http://www.advancedurology.net,26
866,Advanced Urology,https://www.yellowpages.com/search?search_term...,"8700 Beverly BlvdWest Hollywood, CA 90048",(424) 732-1771,"8540 S Sepulveda BlvdLos Angeles, CA 90045","[Physicians & Surgeons, Pediatrics, Medical Ce...",http://www.advancedurology.net,26
867,Assal Hakhamjani,https://www.yellowpages.com/search?search_term...,"2990 S Sepulveda Blvd Ste 206Los Angeles, CA 9...",(424) 274-3260,"8525 W Pico BlvdLos Angeles, CA 90035","[Physicians & Surgeons, Pediatrics]",,30
868,Advanced Urology,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(424) 732-1771,"8540 S Sepulveda BlvdLos Angeles, CA 90045","[Physicians & Surgeons, Pediatrics, Medical Ce...",http://www.advancedurology.net,26


In [1]:
df=pd.read_csv('/content/pedia4.csv')

NameError: ignored

In [51]:
df.to_csv('pedia4.csv',index=False)

In [ ]:
base_url = "https://www.yellowpages.com"
search_url = f"{base_url}/search?search_terms=doctors&geo_location_terms=washington%20dc&page="

links = []

for page in range(1, 60):  # Assuming there are 5 pages to scrape
    url = search_url + str(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    doctor_links = soup.find_all("div", class_="info-section info-primary")
    for link in doctor_links:
        doctor_link = link.find("a")
        if doctor_link:
            href = doctor_link["href"]
            full_link = base_url + href
            links.append(full_link)

for link in links:
    print(link)


https://www.yellowpages.com/washington-dc/mip/dr-juanita-rhodes-dds-pc-8021541?lid=1002067433397
https://www.yellowpages.com/washington-dc/mip/elaine-ellis-center-of-health-471936532?lid=1000439649188
https://www.yellowpages.com/washington-dc/mip/sexter-joanna-md-467826313
https://www.yellowpages.com/washington-dc/mip/melvin-d-gerald-other-462818331
https://www.yellowpages.com/washington-dc/mip/vanvleck-carolyn-466601243
https://www.yellowpages.com/washington-dc/mip/palumbo-francis-466601496
https://www.yellowpages.com/washington-dc/mip/dr-michelle-d-barnes-marshall-md-21565536
https://www.yellowpages.com/washington-dc/mip/evan-nadler-md-479514489
https://www.yellowpages.com/washington-dc/mip/isabel-c-gottron-md-479438466
https://www.yellowpages.com/washington-dc/mip/coyle-james-c-do-541754623
https://www.yellowpages.com/washington-dc/mip/dr-helene-h-felman-md-545041164
https://www.yellowpages.com/washington-dc/mip/sun-kim-562873871
https://www.yellowpages.com/washington-dc/mip/jane-wu

In [6]:
import pandas as pd

In [7]:
df=pd.read_csv('/content/doctor_links.csv')
df.head()

FileNotFoundError: ignored

In [ ]:
len(df['Links'])

1770

In [5]:
from concurrent.futures import ThreadPoolExecutor
import requests
from bs4 import BeautifulSoup

In [ ]:

names = []
addresses = []
phone_numbers = []
personal_pages = []

for link in links:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract the doctor's name
    name = soup.find("h1", class_="dockable business-name").text.strip() if name else ""
    names.append(name)

    # Extract the address
    address = soup.find("span", class_="address").text.strip() if address else ""
    addresses.append(address)

    # Extract the phone number
    phone_number = soup.find("a", class_="phone dockable").text.strip() if phone_number else ""
    phone_numbers.append(phone_number)


    personal_page =soup.find("a", class_="website-link dockable")["href"] if personal_page else ""
    personal_pages.append(personal_page)

    print(name)
    print(addresses)
    print(phone_numbers)
    print(personal_pages)



Dr Juanita Rhodes, DDS, PC
['1647 Benning Rd NE Ste 305Washington, DC 20002']
['(202) 798-7778']
['http://www.drjrhodes.com']
Elaine Ellis Center of Health
['1647 Benning Rd NE Ste 305Washington, DC 20002', '1627 Kenilworth Ave. NEWashington, DC 20019']
['(202) 798-7778', '(202) 803-2340']
['http://www.drjrhodes.com', 'http://www.eechealth.com']
Sexter, Joanna, MD
['1647 Benning Rd NE Ste 305Washington, DC 20002', '1627 Kenilworth Ave. NEWashington, DC 20019', '4900 Massachusetts Ave NWWashington, DC 20016']
['(202) 798-7778', '(202) 803-2340', '(202) 966-5000']
['http://www.drjrhodes.com', 'http://www.eechealth.com', 'http://www.springvalleypediatrics.net']
Melvin D Gerald, Other
['1647 Benning Rd NE Ste 305Washington, DC 20002', '1627 Kenilworth Ave. NEWashington, DC 20019', '4900 Massachusetts Ave NWWashington, DC 20016', '1160 Varnum St NE Ste 117Washington, DC 20017']
['(202) 798-7778', '(202) 803-2340', '(202) 966-5000', '(202) 832-7007']
['http://www.drjrhodes.com', 'http://www.

TypeError: ignored

In [ ]:
p_links=[]
for link in links:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, "html.parser")

    personal_link = soup.find("a", class_="website-link dockable")
    if personal_link:
        href = personal_link["href"]
        p_links.append(href)
        print(full_personal_link)

https://www.yellowpages.comhttp://www.drjrhodes.com
https://www.yellowpages.comhttp://www.eechealth.com
https://www.yellowpages.comhttp://www.springvalleypediatrics.net
https://www.yellowpages.comhttp://www.geraldfamilycare.com
https://www.yellowpages.comhttp://www.springvalleypediatrics.net
https://www.yellowpages.comhttp://www.springvalleypediatrics.net
https://www.yellowpages.comhttp://www.mbarnesmarshallmdpc.com
https://www.yellowpages.comhttps://childrensnational.org/visit/find-a-provider/evan-nadler
https://www.yellowpages.comhttp://kp.org/doctor/isabelgottron
https://www.yellowpages.comhttps://www.kp.org/doctor/jamescoyle
https://www.yellowpages.comhttps://www.kp.org/doctor/helenefelman
https://www.yellowpages.comhttps://locations.myeyedr.com/dc/washington/1744-l-st-nw
https://www.yellowpages.comhttp://www.pedaids.org
https://www.yellowpages.comhttps://childrensnational.org/visit/find-a-provider/ellen-obrien
https://www.yellowpages.comhttps://www.kp.org/doctor/keithegan
https://

KeyboardInterrupt: ignored

In [ ]:
import csv

In [ ]:
filename = "doctor_links.csv"
with open(filename, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Links"])  # Write header
    writer.writerows([[link] for link in links])

In [ ]:
doctor_page_links = [link["href"] for link in doctor_links]

for link in doctor_page_links:
    print(link)


In [4]:
base_url = "https://www.yellowpages.com/search?search_terms=pediatrician&geo_location_terms=Los+Angeles%2C+CA"
search_url = f"{base_url}/search?search_terms=doctors&geo_location_terms=washington%20dc&page="

links = []

for page in range(1, 30):  # Assuming there are 5 pages to scrape
    url = search_url + str(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    doctor_links = soup.find_all("div", class_="info-section info-primary")
    for link in doctor_links:
        doctor_link = link.find("a")
        if doctor_link:
            href = doctor_link["href"]
            full_link = base_url + href
            links.append(full_link)

for link in links:
    print(link)

https://www.yellowpages.com/search?search_terms=pediatrician&geo_location_terms=Los+Angeles%2C+CA/los-angeles-ca/mip/advanced-urology-541807926?lid=1002150403102
https://www.yellowpages.com/search?search_terms=pediatrician&geo_location_terms=Los+Angeles%2C+CA/los-angeles-ca/mip/mohsen-m-hamza-m-d-neurology-center-for-adults-children-21959437?lid=171627628
https://www.yellowpages.com/search?search_terms=pediatrician&geo_location_terms=Los+Angeles%2C+CA/los-angeles-ca/mip/eisner-pediatric-family-medical-center-467566758?lid=1000909983377
https://www.yellowpages.com/search?search_terms=pediatrician&geo_location_terms=Los+Angeles%2C+CA/los-angeles-ca/mip/california-kids-pediatrics-473948651
https://www.yellowpages.com/search?search_terms=pediatrician&geo_location_terms=Los+Angeles%2C+CA/los-angeles-ca/mip/abc-pediatrics-inc-455359700
https://www.yellowpages.com/search?search_terms=pediatrician&geo_location_terms=Los+Angeles%2C+CA/los-angeles-ca/mip/dr-audrey-griesbach-md-7438796
https://ww

In [5]:
len(links)

870

In [8]:
import pandas as pd
import concurrent.futures

In [9]:
import multiprocessing

In [31]:
personal_pages = []
for link in df['links']:
    response = requests.get(link)
    soup = BeautifulSoup(response.content,'html.parser')  # Assuming 'link' contains the HTML content of the page
    personal_page_element = soup.find("div", class_="links")
    page = personal_page_element.find("a")["href"] if personal_page_element else ""
    personal_pages.append(page)

print(personal_page)

TypeError: ignored

In [32]:
def process_link(link):
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    try:
        personal_page_element = soup.find("div", class_="links")
        page = personal_page_element.find("a")["href"] if personal_page_element else ""
    except (AttributeError, TypeError):
        page = ""
    return page

def process_batch(links):
    personal_pages = []
    for link in links:
        page = process_link(link)
        personal_pages.append(page)
    return personal_pages

# Set the number of processes to use
num_processes = multiprocessing.cpu_count()

# Split the links into batches
batch_size = 10
link_batches = [df['links'][i:i+batch_size] for i in range(0, len(df['links']), batch_size)]

# Create a multiprocessing pool
pool = multiprocessing.Pool(processes=num_processes)

# Process the link batches in parallel
results = pool.map(process_batch, link_batches)

# Combine the results from all batches
personal_pages = [page for batch_result in results for page in batch_result]

print(personal_pages)


['http://www.advancedurology.net', 'http://www.westside-pediatrics.com', 'http://www.pedcenter.org', 'http://www.calkidspeds.com', 'http://www.abcpediatrics.com', 'http://www.calkidspeds.com', 'http://www.advancedurology.net', 'http://www.advancedurology.net', 'http://www.advancedurology.net', 'http://www.pediatricassociatesofdayton.com/', 'http://www.calkidspeds.com', '', '', '', '', '', 'http://www.pedcenter.org', 'http://www.advancedurology.net', '', '', '', 'http://www.pedcenter.org', '', '', 'https://www.optum.com/care/providers/ca/los-angeles/christine-kao-do.html', 'https://www.optum.com/care/providers/ca/los-angeles/christine-kao-do.html', 'http://www.advancedurology.net', '', 'https://www.uclahealth.org/providers/nicole-militello?utm_source=ExtNet&utm_medium=Yext', '', '', 'http://www.advancedurology.net', 'https://www.cedars-sinai.edu', 'https://www.cedars-sinai.edu', 'https://healthy.kaiserpermanente.org/health/poc?uri=my-doctor:dynamic&id=4645707&kpSearch=', 'http://www.ped

In [78]:
print(len(personal_page))

870


In [46]:
names = []
addresses = []
phone_numbers = []


batch_size = 5  # Number of links to process in each batch
num_processes = 10  # Number of processes to use

# Define a function to process a batch of links
def process_batch(batch_links):
    batch_names = []
    batch_addresses = []
    batch_phone_numbers = []
  

    for link in batch_links:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract the doctor's name
        name_element = soup.find("a", class_="business-name")
        name = name_element.text.split(', ')[0] if name_element else ""

        # Extract the address
        address_element = soup.find("div", class_="adr")
        address = address_element.get_text(strip=True) if address_element else ""

        # Extract the phone number
        phone_number_element = soup.find("div", class_="phones phone primary")
        phone_number = phone_number_element.text.strip() if phone_number_element else ""


        batch_names.append(name)
        batch_addresses.append(address)
        batch_phone_numbers.append(phone_number)
       

    return batch_names, batch_addresses, batch_phone_numbers

# Split the links into batches
batches = [links[i:i+batch_size] for i in range(0, len(links), batch_size)]

# Create a ProcessPoolExecutor
with concurrent.futures.ProcessPoolExecutor(max_workers=num_processes) as executor:
    # Submit tasks for each batch of links
    results = [executor.submit(process_batch, batch) for batch in batches]

    # Process the results as they complete
    for future in concurrent.futures.as_completed(results):
        # Retrieve the results of a completed task
        batch_names, batch_addresses, batch_phone_numbers= future.result()

        # Append the batch results to the respective lists
        names.extend(batch_names)
        addresses.extend(batch_addresses)
        phone_numbers.extend(batch_phone_numbers)
        

        print(batch_names)
        print(batch_addresses)
        print(batch_phone_numbers)
      

print(names)
print(addresses)
print(phone_numbers)


['California Kids Pediatrics', 'Benjamin Tiet', 'Benjamin Tiet', 'Benjamin Tiet', 'Benjamin Tiet']
['2990 S Sepulveda Blvd Ste 206Los Angeles, CA 90064', '10833 Le Conte Ave B2-427  MdccLos Angeles, CA 90095', '10833 Le Conte Ave B2-427  MdccLos Angeles, CA 90095', '10833 Le Conte Ave B2-427  MdccLos Angeles, CA 90095', '10833 Le Conte Ave B2-427  MdccLos Angeles, CA 90095']
['(424) 277-2020', '(310) 825-5296', '(310) 825-5296', '(310) 825-5296', '(310) 825-5296']
['California Kids Pediatrics', 'Advanced Urology', 'Advanced Urology', 'Advanced Urology', 'Pediatric Associates of Dayton Inc']
['2990 S Sepulveda Blvd Ste 206Los Angeles, CA 90064', '8540 S Sepulveda BlvdLos Angeles, CA 90045', '8540 S Sepulveda BlvdLos Angeles, CA 90045', '8540 S Sepulveda BlvdLos Angeles, CA 90045', '3782 W Martin Luther King Jr BlvdLos Angeles, CA 90008']
['(424) 277-2020', '(424) 732-1771', '(424) 732-1771', '(424) 732-1771', '(323) 421-2987']
['Benjamin Tiet', 'Eisner Pediatric & Family Medical Center'

In [79]:
df=pd.DataFrame({'Name':names,'links':links,'address':addresses,'Phone_number':phone_numbers,'Personal_page':personal_page})

In [21]:
df=pd.read_csv('/content/pedia1.csv')

In [22]:
df

,Name,links,address,Phone_number
0,California Kids Pediatrics,https://www.yellowpages.com/search?search_term...,"2990 S Sepulveda Blvd Ste 206Los Angeles, CA 9...",(424) 277-2020
1,Benjamin Tiet,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(310) 825-5296
2,Benjamin Tiet,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(310) 825-5296
3,Benjamin Tiet,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(310) 825-5296
4,Benjamin Tiet,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(310) 825-5296
...,...,...,...,...
865,Dr. Eva E Rodriguez,https://www.yellowpages.com/search?search_term...,"1403 Lomita BlvdHarbor City, CA 90710",(310) 325-4344
866,Dr. Yana Tavyev,https://www.yellowpages.com/search?search_term...,"8700 Beverly BlvdWest Hollywood, CA 90048",(310) 967-1884
867,California Kids Pediatrics,https://www.yellowpages.com/search?search_term...,"2990 S Sepulveda Blvd Ste 206Los Angeles, CA 9...",(424) 277-2020
868,Benjamin Tiet,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(310) 825-5296


In [51]:
df.to_csv('pedia3.csv',index=False)

1. Seriel Number
2. Base Link from the Yellow Pages
3. Doctors/Business Name
4. Individual Profile Link fron the Yellow Pages
5. Categories
6. Contact Number
7. Fax Number
8. Email Id
9. Address
10. City
11. Years in Business
12. Individual Website Link
13. Facebook
14. Instagram
15. Twitter
16. Youtube
17. LinkdIn

In [23]:
cat = []
links = df['links'].tolist()
batch_size = 5
num_processes = 10

# Define a function to process a batch of links
def process_batch(batch_links):
    batch_categories = []
    
    for link in batch_links:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, "html.parser")
        cat_ele = soup.find("div", class_="categories")
        categories = [category.get_text(strip=True) for category in cat_ele.find_all("a")] if cat_ele else []
        batch_categories.append(categories)

    return batch_categories

# Split the links into batches
batches = [links[i:i+batch_size] for i in range(0, len(links), batch_size)]

# Create a ProcessPoolExecutor
with concurrent.futures.ProcessPoolExecutor(max_workers=num_processes) as executor:
    # Submit tasks for each batch of links
    results = [executor.submit(process_batch, batch) for batch in batches]

    # Process the results as they complete
    for future in concurrent.futures.as_completed(results):
        # Retrieve the results of a completed task
        batch_categories = future.result()

        # Append the batch results to the cat list
        cat.extend(batch_categories)

print(cat)



[['Physicians & Surgeons, Pediatrics', 'Physicians & Surgeons'], ['Physicians & Surgeons, Pediatrics', 'Physicians & Surgeons'], ['Physicians & Surgeons, Pediatrics', 'Physicians & Surgeons, Neurology', 'Physicians & Surgeons, Pediatrics-Neurology'], ['Physicians & Surgeons, Pediatrics', 'Physicians & Surgeons', 'Medical Clinics', 'Clinics'], ['Physicians & Surgeons, Pediatrics', 'Physicians & Surgeons'], ['Physicians & Surgeons, Pediatrics', 'Physicians & Surgeons'], ['Physicians & Surgeons, Pediatrics', 'Medical Centers', 'Physicians & Surgeons, Family Medicine & General Practice'], ['Physicians & Surgeons, Pediatrics', 'Physicians & Surgeons, Sexual Dysfunction', 'Medical Centers'], ['Physicians & Surgeons, Pediatrics', 'Physicians & Surgeons'], ['Physicians & Surgeons, Pediatrics', 'Physicians & Surgeons'], ['Physicians & Surgeons, Pediatrics', 'Physicians & Surgeons', 'Clinics'], ['Physicians & Surgeons, Pediatrics', 'Physicians & Surgeons, Sexual Dysfunction', 'Medical Centers'],

In [9]:
years = []
links = df['links'].tolist()
batch_size = 5
num_processes = 10

# Define a function to process a batch of links
def process_batch(batch_links):
    batch_categories = []
    
    for link in batch_links:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, "html.parser")
        cat_ele = soup.find("div", class_="number")
        categories = cat_ele.text if cat_ele else ""
        batch_categories.append(int(categories) if categories else 0)

    return batch_categories

# Split the links into batches
batches = [links[i:i+batch_size] for i in range(0, len(links), batch_size)]

# Create a ProcessPoolExecutor
with concurrent.futures.ProcessPoolExecutor(max_workers=num_processes) as executor:
    # Submit tasks for each batch of links
    results = [executor.submit(process_batch, batch) for batch in batches]

    # Process the results as they complete
    for future in concurrent.futures.as_completed(results):
        # Retrieve the results of a completed task
        batch_categories = future.result()

        # Append the batch results to the cat list
        years.extend(batch_categories)

        print(years)

[22, 103, 24, 103, 59]
[22, 103, 24, 103, 59, 26, 6, 103, 44, 15]
[22, 103, 24, 103, 59, 26, 6, 103, 44, 15, 22, 22, 31, 46, 22]
[22, 103, 24, 103, 59, 26, 6, 103, 44, 15, 22, 22, 31, 46, 22, 59, 26, 47, 44, 27]
[22, 103, 24, 103, 59, 26, 6, 103, 44, 15, 22, 22, 31, 46, 22, 59, 26, 47, 44, 27, 44, 47, 43, 47, 29]
[22, 103, 24, 103, 59, 26, 6, 103, 44, 15, 22, 22, 31, 46, 22, 59, 26, 47, 44, 27, 44, 47, 43, 47, 29, 44, 22, 22, 22, 22]
[22, 103, 24, 103, 59, 26, 6, 103, 44, 15, 22, 22, 31, 46, 22, 59, 26, 47, 44, 27, 44, 47, 43, 47, 29, 44, 22, 22, 22, 22, 26, 46, 38, 38, 38]
[22, 103, 24, 103, 59, 26, 6, 103, 44, 15, 22, 22, 31, 46, 22, 59, 26, 47, 44, 27, 44, 47, 43, 47, 29, 44, 22, 22, 22, 22, 26, 46, 38, 38, 38, 44, 26, 26, 26, 34]
[22, 103, 24, 103, 59, 26, 6, 103, 44, 15, 22, 22, 31, 46, 22, 59, 26, 47, 44, 27, 44, 47, 43, 47, 29, 44, 22, 22, 22, 22, 26, 46, 38, 38, 38, 44, 26, 26, 26, 34, 29, 26, 22, 15, 30]
[22, 103, 24, 103, 59, 26, 6, 103, 44, 15, 22, 22, 31, 46, 22, 59, 26, 47

In [10]:
print(len(years))

870


In [25]:
df['Categories']=cat

In [46]:
df['Years_in_business']=years


In [47]:
df['Personal_website']= personal_pages

In [43]:
personal_pages

['http://www.advancedurology.net',
 'http://www.westside-pediatrics.com',
 'http://www.pedcenter.org',
 'http://www.calkidspeds.com',
 'http://www.abcpediatrics.com',
 'http://www.calkidspeds.com',
 'http://www.advancedurology.net',
 'http://www.advancedurology.net',
 'http://www.advancedurology.net',
 'http://www.pediatricassociatesofdayton.com/',
 'http://www.calkidspeds.com',
 '',
 '',
 '',
 '',
 '',
 'http://www.pedcenter.org',
 'http://www.advancedurology.net',
 '',
 '',
 '',
 'http://www.pedcenter.org',
 '',
 '',
 'https://www.optum.com/care/providers/ca/los-angeles/christine-kao-do.html',
 'https://www.optum.com/care/providers/ca/los-angeles/christine-kao-do.html',
 'http://www.advancedurology.net',
 '',
 'https://www.uclahealth.org/providers/nicole-militello?utm_source=ExtNet&utm_medium=Yext',
 '',
 '',
 'http://www.advancedurology.net',
 'https://www.cedars-sinai.edu',
 'https://www.cedars-sinai.edu',
 'https://healthy.kaiserpermanente.org/health/poc?uri=my-doctor:dynamic&id=4

In [49]:
df.drop('Personal_page', axis=1, inplace =True)

In [50]:
df

,Name,links,address,Phone_number,Categories,Years_in_business,Personal_website
0,California Kids Pediatrics,https://www.yellowpages.com/search?search_term...,"2990 S Sepulveda Blvd Ste 206Los Angeles, CA 9...",(424) 277-2020,"['Physicians & Surgeons, Pediatrics', 'Physici...",22,http://www.advancedurology.net
1,Benjamin Tiet,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(310) 825-5296,"['Physicians & Surgeons, Pediatrics', 'Physici...",103,http://www.westside-pediatrics.com
2,Benjamin Tiet,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(310) 825-5296,"['Physicians & Surgeons, Pediatrics', 'Physici...",24,http://www.pedcenter.org
3,Benjamin Tiet,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(310) 825-5296,"['Physicians & Surgeons, Pediatrics', 'Physici...",103,http://www.calkidspeds.com
4,Benjamin Tiet,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(310) 825-5296,"['Physicians & Surgeons, Pediatrics', 'Physici...",59,http://www.abcpediatrics.com
...,...,...,...,...,...,...,...
865,Dr. Eva E Rodriguez,https://www.yellowpages.com/search?search_term...,"1403 Lomita BlvdHarbor City, CA 90710",(310) 325-4344,"['Physicians & Surgeons, Pediatrics', 'Medical...",26,http://www.comprehensive-urology.com
866,Dr. Yana Tavyev,https://www.yellowpages.com/search?search_term...,"8700 Beverly BlvdWest Hollywood, CA 90048",(310) 967-1884,"['Physicians & Surgeons, Pediatrics', 'Medical...",26,http://www.advancedurology.net
867,California Kids Pediatrics,https://www.yellowpages.com/search?search_term...,"2990 S Sepulveda Blvd Ste 206Los Angeles, CA 9...",(424) 277-2020,[],30,
868,Benjamin Tiet,https://www.yellowpages.com/search?search_term...,"10833 Le Conte Ave B2-427 MdccLos Angeles, CA...",(310) 825-5296,"['Physicians & Surgeons, Pediatrics', 'Medical...",26,http://www.advancedurology.net


In [17]:
import re

In [23]:
facebook_links = []

for personal_page in df['Personal_page'][:5]:
    if personal_page:
        response = requests.get(personal_page)
        soup = BeautifulSoup(response.content, "html.parser")

        facebook_link_element = soup.find('a', href=re.compile(r'facebook.com'))
        facebook_link = facebook_link_element['href'] if facebook_link_element else ""
        facebook_links.append(facebook_link)
    else:
        facebook_links.append("")
print(facebook_links)

['https://www.facebook.com/uclahealth', 'https://www.facebook.com/uclahealth', 'https://www.facebook.com/uclahealth', 'https://www.facebook.com/uclahealth', 'https://www.facebook.com/uclahealth']


In [18]:
linkedin_links = []

for personal_page in df['Personal_page'][:5]:
    if personal_page:
        response = requests.get(personal_page)
        soup = BeautifulSoup(response.content, "html.parser")

        linkedin_link_element = soup.find('a', href=re.compile(r'linkedin.com'))
        linkedin_link = linkedin_link_element['href'] if linkedin_link_element else ""
        linkedin_links.append(linkedin_link)
    else:
        linkedin_links.append("")
        
print(linkedin_links)

['https://linkedin.com/company/ucla-health', 'https://linkedin.com/company/ucla-health', 'https://linkedin.com/company/ucla-health', 'https://linkedin.com/company/ucla-health', 'https://linkedin.com/company/ucla-health']


In [20]:
import re

instagram_links = []
email_addresses = []
twitter_links = []

for personal_page in df['Personal_page'][50:55]:
    if personal_page:
        response = requests.get(personal_page)
        soup = BeautifulSoup(response.content, "html.parser")

        # Instagram link
        instagram_link_element = soup.find('a', href=re.compile(r'instagram.com'))
        instagram_link = instagram_link_element['href'] if instagram_link_element else ""
        instagram_links.append(instagram_link)

        # Email address
        email_element = soup.find('a', href=re.compile(r'mailto:'))
        email_address = email_element['href'].replace('mailto:', '') if email_element else ""
        email_addresses.append(email_address)

        # Twitter link
        twitter_link_element = soup.find('a', href=re.compile(r'twitter.com'))
        twitter_link = twitter_link_element['href'] if twitter_link_element else ""
        twitter_links.append(twitter_link)
    else:
        instagram_links.append("")
        email_addresses.append("")
        twitter_links.append("")

print("Instagram Links:", instagram_links)
print("Email Addresses:", email_addresses)
print("Twitter Links:", twitter_links)


Instagram Links: ['https://instagram.com/uclahealth', 'https://instagram.com/uclahealth', 'https://instagram.com/uclahealth', 'https://instagram.com/uclahealth', 'https://instagram.com/uclahealth']
Email Addresses: ['', '', '', '', '']
Twitter Links: ['https://twitter.com/uclahealth', 'https://twitter.com/uclahealth', 'https://twitter.com/uclahealth', 'https://twitter.com/uclahealth', 'https://twitter.com/uclahealth']
